In [61]:
#import webapp2
import logging
#from google.appengine.ext import ndb
#from google.appengine.api import taskqueue
#from google.appengine.ext import db

from google.cloud import ndb
from google.cloud import ndb


class User_Profile_Prefemails(ndb.Model):
    #key = preferred email address
    original_email = ndb.StringProperty()
    date_created = ndb.DateTimeProperty(auto_now_add=True)
    date_updated = ndb.DateTimeProperty(auto_now=True)

class UserActivity(ndb.Model):
    #key = user email 
    email_pref = ndb.StringProperty() #preferred email address    
   
        
class PrefEmailCheck():#webapp2.RequestHandler):
    get_methods = {
        '/admin/user/test_create_dummy_UserActivity_records': 'test_create_dummy_UserActivity_records',
        '/admin/user/pref-email-update-task' : 'pref_email_update_task',
    }

    post_methods = {
        '/admin/user/pref-email-update-task' : 'pref_email_update_task',
    }

    # Methods Definitions  
    def __init__(self): #, request='', response=''):  
        super(PrefEmailCheck, self).__init__() #request, response)
        self.request = 'request'
        self.response = 'response'
        self.GETSTR = 'get'
        self.POSTSTR = 'post'
        return 
    
    def get(self):
        logging.info('In get method')
        self.process_url(self.GETSTR)
        return

    def post(self):
        logging.info('In Post method')
        self.process_url(self.POSTSTR) # Correction 1
        return

    def process_url(self, source):
        #validate URL
        rcvd_url = self.request.path
        logging.info('Rcvd url = %s', rcvd_url)
        if source == self.GETSTR:
            func_map = self.get_methods
        else:
            func_map = self.post_methods
            
        #secObj = sec.SecurityHandler()
        #if secObj.validate_url(rcvd_url):
            #get destination method using key-value pair
            dest_method = func_map.get(rcvd_url, None) # Correction 2
            logging.info("dest_method is {}".format(dest_method))
            if dest_method: 
                func = getattr(self, dest_method, None)
                logging.info("func is {}".format(func))
                if func:
                    func()
                    logging.info('Finished %s method', source)        
                    return
            #Invalid url
            else:
                pass #case handled below

        #Invalid URL
        #err_obj = err.ErrorHandler()
        #err_obj.handle_invalid_url(rcvd_url)
        #self.error(404)
        logging.info('Finished %s method with error', source)        
        return

    def test_create_dummy_UserActivity_records(self):
        #In this function, create 10-15 dummy records in UserActivity database for testing purposes
        logging.info("In test_create_dummy_UserActivity_records")

        ## LOGIC GOES HERE ##    
        UserActivity(email_pref = "Abc@gmail.com").put()        
        UserActivity(email_pref = "Ram@gmail.com").put()                
        UserActivity(email_pref = "Ice@gmail.com").put()                
        UserActivity(email_pref = "Rock@gmail.com").put()       
        UserActivity(email_pref = "Paige@gmail.com").put()                
        UserActivity(email_pref = "Shyam@gmail.com").put()                
        UserActivity(email_pref = "Ajay@gmail.com").put()               
        UserActivity(email_pref = "Suresh@gmail.com").put()                
        UserActivity(email_pref = "Kamal@gmail.com").put()                
        UserActivity(email_pref = "Venkat@gmail.com").put()        
        UserActivity(email_pref = "Sameer@gmail.com").put()        
        UserActivity(email_pref = "Venkat@gmail.com").put()        
        UserActivity(email_pref = "Darshan@gmail.com").put()        
        UserActivity(email_pref = "Navjot@gmail.com").put()
        UserActivity(email_pref = "Virat@gmail.com").put()
        
        logging.info("Exiting test_create_dummy_UserActivity_records")
        return 
    
    def pref_email_update_task(self):
        logging.info("In pref_email_update_task")

        cursor_from_url = self.request.get('cursor')
        logging.info("Cursor in start is: {}".format(cursor_from_url))

        gql_query_text = 'Select * From User_Activity Where email_pref != NULL' #Define query conditions here
        
        #Execute query
        query_o = db.GqlQuery(gql_query_text)
        if cursor_from_url:
            query_o.with_cursor(cursor_from_url)
            BATCH_SIZE = 2
        else:
            BATCH_SIZE = 1
        results = query_o.fetch(BATCH_SIZE)

        if not results:
            logging.info("pref_email_update_task: all done")
            return  

        ## ADD LOGIC TO RUN FOR LOOP AND PUT DATA IN User_Profile_Prefemails database ##
        for row in results:
            User_Profile_Prefemails(original_email = row).put()
            email = is_pref_email(User_Profile_Prefemails.original_email)            
        cursor = query_o.cursor()
        taskqueue.add(url='/admin/user/pref-email-update-task', params={'cursor': cursor})
        logging.info("Exiting pref_email_update_task")
        return
    
    def is_pref_email(self, email):
        #Check if this email id exists in the PrefEmail database, if so, return the original_email        
        if  self.email:
            self.email = User_Profile_Prefemails.original_email 
            return original_email
        else:
            logging.info("Email does not exist")
            return None        
        return 

#app = webapp2.WSGIApplication([
#    ('/admin/user/test_create_dummy_UserActivity_records', PrefEmailCheck),
 #   ('/admin/user/pref-email-update-task', PrefEmailCheck),
  #  ], debug=True)



In [62]:
s1  = PrefEmailCheck()
s2  = UserActivity()
s3 = PrefEmailCheck()
s3.test_create_dummy_UserActivity_records()

ContextError: No current context. NDB calls must be made in context established by google.cloud.ndb.Client.context.

In [56]:
from google.cloud import datastore
from google.cloud import ndb

client = ndb.Client()

class Test(ndb.Model):
    name = ndb.StringProperty()  
    
    def test(input_name):
        print('Adding {}'.format(input_name))
        with client.context():
            t = Test(
                name=input_name,             
            )
            t.put()        
        print('Added {}'.format(input_name))
        
        if __name__ == "__main__":
            import threading
            threading.Thread(target=test, args=['John']).start()
            threading.Thread(target=test, args=['Bob']).start()

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [55]:
from google.cloud import storage
import configparser
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import ndb
from google.cloud import datastore
credentials = service_account.Credentials.from_service_account_file\
              ('C:/Users/shiva/AppData/Local/Google/CloudSDK/jinja2_app/Projects/Step3-gql/Project#1/mycredentials.json')
config = configparser.ConfigParser()
config.read("C:/Users/shiva/AppData/Local/Google/CloudSDK/jinja2_app/Projects/Step3-gql/Project#1/config.txt")
project_id = config.get("configuration","project_id") 
client = ndb.Client()
    
class Test(ndb.Model):
    name = ndb.StringProperty()
    def test(input_name):    
        print('Adding {}'.format(input_name))
        with client.context():
            t = Test(
                name=input_name,             
            ) 
            t.put()        
            print('Added {}'.format(input_name))
if __name__ == "__main__":
    import threading
    threading.Thread(target=test, args=['John']).start()
    threading.Thread(target=test, args=['Bob']).start()
    print("Hello cred",credentials)

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [53]:
implicit()

Hello cred <google.oauth2.service_account.Credentials object at 0x000002019F91A348>
